In [1]:
import numpy as np
print("Succesfully imported %s -- Version: %s"%(np.__name__,np.__version__))
import scipy
print("Succesfully imported %s -- Version: %s"%(scipy.__name__,scipy.__version__))
import sympy as sym 
print("Succesfully imported %s -- Version: %s"%(sym.__name__,sym.__version__))

import matplotlib.pyplot as plt

Succesfully imported numpy -- Version: 1.19.5
Succesfully imported scipy -- Version: 1.6.0
Succesfully imported sympy -- Version: 1.7.1


# Linear Shape Functions

## Reference Element

In [10]:
sym.var('x')
sym.var('y')

phi1 = 1-x-y
phi2 = x
phi3 = y

phi    = sym.Matrix([phi1, phi2, phi3])
dphi_x = sym.diff(phi, x);
dphi_y = sym.diff(phi, y);

A = sym.integrate(sym.integrate(dphi_x * sym.Transpose(dphi_x) + dphi_y * sym.Transpose(dphi_y), (y, 1 - x)), (x, 0.,1))
M = sym.integrate(sym.integrate(phi*sym.Transpose(phi), (y, 1 - x)), (x, 0.,1))

print('contribution to the stiffness matrix'), sym.pprint(A)
print('contribution to the mass matrix'), sym.pprint(M)

contribution to the stiffness matrix
⎡ 1    -1/2  -1/2⎤
⎢                ⎥
⎢-1/2  1/2    0  ⎥
⎢                ⎥
⎣-1/2   0    1/2 ⎦
contribution to the mass matrix
⎡1/12  1/24  1/24⎤
⎢                ⎥
⎢1/24  1/12  1/24⎥
⎢                ⎥
⎣1/24  1/24  1/12⎦


(None, None)

## Direct Integration of Geometric Element

In [188]:
sym.var('x')
sym.var('y')

x1 = 1.2; y1 = 1;
x2 = 1.5; y2 = 1.25;
x3 = 1;   y3 = 1.5;

Emat = sym.Matrix([[x1, y1, 1], [x2, y2, 1], [x3, y3, 1]]).inv();

phi = sym.transpose(Emat) * sym.Matrix([[x], [y], [1]])
dphi_x = sym.diff(phi, x);
dphi_y = sym.diff(phi, y);

phi

Matrix([
[-1.25*x - 2.5*y + 5.0],
[  2.5*x + 1.0*y - 4.0],
[      -1.25*x + 1.5*y]])

In [136]:
area_id = (x1 - x2)*(y1 - y3) - (x1 - x3)*(y1 - y2);
area_id = abs(area_id) / 2;

A = area_id * (dphi_x * sym.Transpose(dphi_x) + dphi_y * sym.Transpose(dphi_y))
M = 0 # Cannot easily integrate this over the element..

A

Matrix([
[ 0.78125, -0.5625, -0.21875],
[ -0.5625,   0.725,  -0.1625],
[-0.21875, -0.1625,  0.38125]])

## Current Calculation Code

In [129]:
Dmat = sym.Matrix([Emat[0, :], Emat[1, :], [0, 0, 0]])
A = area_id * sym.Transpose(Dmat) * Dmat

A

Matrix([
[ 0.78125, -0.5625, -0.21875],
[ -0.5625,   0.725,  -0.1625],
[-0.21875, -0.1625,  0.38125]])

## Affine Mapping

In [147]:
sym.var('x')
sym.var('y')

y

In [282]:
phi = sym.Transpose(Emat) * sym.Matrix([[x], [y], [1]])

X = sym.Matrix([[0, 0]]);
Xr = sym.Matrix([[0, 0], [1, 0], [0, 1]]);
for i in range(0, 3):
    X += phi[i] * Xr[i,:]
    
grad_phi1 = -sym.Matrix([[sym.diff(X[0], x) + sym.diff(X[1], x)], [sym.diff(X[0], y) + sym.diff(X[1], y)]])
grad_phi2 = sym.Matrix([[sym.diff(X[0], x)], [sym.diff(X[0], y)]])
grad_phi3 = sym.Matrix([[sym.diff(X[1], x)], [sym.diff(X[1], y)]])

grad_phi = sym.Matrix([[grad_phi1.T], [grad_phi2.T], [grad_phi3.T]])

A = 2 * area_id * sym.integrate(sym.integrate(grad_phi * grad_phi.T, (x, 0, 1 - y)), (y, 0, 1)); A

Matrix([
[ 0.78125, -0.5625, -0.21875],
[ -0.5625,   0.725,  -0.1625],
[-0.21875, -0.1625,  0.38125]])

In [278]:
phiR = sym.Matrix([[1 - x - y], [x], [y]])

M = 2 * area_id * sym.integrate(sym.integrate(phiR * phiR.T, (x, 0, 1 - y)), (y, 0, 1)); M

Matrix([
[ 0.0166666666666667, 0.00833333333333333, 0.00833333333333333],
[0.00833333333333333,  0.0166666666666667, 0.00833333333333333],
[0.00833333333333333, 0.00833333333333333,  0.0166666666666667]])

In [277]:
grad_phi 

Matrix([
[-1.25, -2.5],
[  2.5,  1.0],
[-1.25,  1.5]])

In [288]:
X

Matrix([[2.5*x + 1.0*y - 4.0, -1.25*x + 1.5*y]])